# Calculate Forgetting Rate (FR) Independent Version

This notebook is for calculating Forgetting Rate (FR) metric (Independent Version) of continual learning experiments. We support calculating for multiple experiments (with respect to one reference joint learning experiment) in an output folder at the same time.

Please specify all the settings (the cells tagged "Please Specify") following the instructions and Hit "Run All". 

## 1. Specify Output Folder

Please specify the output folder path that you want to calcualate FR. 

In [ ]:
group_folder_path: str = "../outputs/til_pmnist_finetuning/"

Please specify the accuracy CSV file of the reference joint learning experiment. 

In [ ]:
ilref_acc_path: str = "../outputs/til_pmnist_independent/acc.csv"

Please specify the accuracy CSV file of the reference random experiment. 

In [ ]:
randref_acc_path: str = "../outputs/random_til_pmnist_random/acc.csv"

Please specify the relative path (with respect to result folder) of accuracy CSV file. Default is `results/acc.csv`.

In [ ]:
acc_relative_path: str = "results/acc.csv"

Please specify the relative path (with respect to result folder) of calculated FR CSV file. Default is `results/fr.csv`.

In [ ]:
fr_relative_path: str = "results/fr.csv"

Building paths...

In [ ]:
import os

output_folder_names = [f for f in os.listdir(group_folder_path) if os.path.isdir(os.path.join(group_folder_path, f))]

## 2. Calculate FR

We read the accuracy CSV files into Pandas Dataframes and calculate FR, then save to CSV files.

In [ ]:
import pandas as pd


for folder_name in output_folder_names:
    folder_path = os.path.join(group_folder_path, folder_name)
    acc_path = os.path.join(folder_path, acc_relative_path)
    fr_path = os.path.join(folder_path, fr_relative_path)

    if not os.path.exists(acc_path):
        print(f"File {acc_path} does not exist. Skipping...")
        continue
    
    acc_df = pd.read_csv(acc_path)
    ilref_acc_df = pd.read_csv(ilref_acc_path)
    randref_acc_df = pd.read_csv(randref_acc_path)
    
    # Get the number of tasks
    num_tasks = acc_df.shape[0]
    
    fr_data = []
    for N in range(1, num_tasks + 1):
        fr_N = 0.0
        for t in range(1, N + 1):
            a_t_N = acc_df.loc[acc_df['after_training_task'] == N, f'test_on_task_{t}'].values
            a_t_I = ilref_acc_df.loc[ilref_acc_df['after_training_task'] == t, f'test_on_task_{t}'].values
            a_t_R = randref_acc_df.loc[randref_acc_df['after_training_task'] == t, f'test_on_task_{t}'].values
            fr_N += (a_t_N - a_t_R) / (a_t_I - a_t_R)
        fr_N /= (N - 1)
        fr_N -= 1
        fr_data.append({'after_training_task': N, 'FR': fr_N})
    
    fr_df = pd.DataFrame(fr_data)

    # Save FR to a CSV file
    fr_df.to_csv(fr_path)
    print(f"Saved FR to {fr_path}")